In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q flwr["simulation"] tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.

In [3]:
import flwr as fl
import tensorflow as tf
import numpy as np
from typing import List, Tuple
from flwr.common import Metrics
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
train_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_train.npy")
train_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_train.npy")
test_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_test.npy")
test_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_test.npy")

In [5]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(6):
  x_train.append(train_data[train_data.shape[0]//6*i:train_data.shape[0]//6*(i+1)])
  y_train.append(train_label[train_label.shape[0]//6*i:train_label.shape[0]//6*(i+1)])
  x_test.append(test_data[test_data.shape[0]//6*i:test_data.shape[0]//6*(i+1)])
  y_test.append(test_label[test_label.shape[0]//6*i:test_label.shape[0]//6*(i+1)])

In [6]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train_cid, y_train_cid, x_test_cid, y_test_cid) -> None:
        self.model = model
        self.x_train_cid, self.y_train_cid = x_train_cid, y_train_cid
        self.x_test_cid, self.y_test_cid = x_test_cid, y_test_cid

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train_cid, self.y_train_cid, epochs=5, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train_cid), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_test_cid, self.y_test_cid, verbose=0)
        return loss, len(self.x_test_cid), {"accuracy": acc}

In [7]:
def client_fn(cid: str) -> fl.client.Client:

    model = Sequential([
      layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
      layers.MaxPool1D(),
      layers.Conv1D(32, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Conv1D(64, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(5 ,activation="softmax")
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

    x_train_cid = x_train[int(cid)]
    y_train_cid = y_train[int(cid)]

    x_test_cid = x_test[int(cid)]
    y_test_cid = y_test[int(cid)]


    return FlowerClient(model, x_train_cid, y_train_cid, x_test_cid, y_test_cid)

In [8]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:

    print(metrics)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]


    return {"accuracy": sum(accuracies) / sum(examples)}

In [9]:
#created for initial parameters
model = Sequential([
  layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
  layers.MaxPool1D(),
  layers.Conv1D(32, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Conv1D(64, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(5 ,activation="softmax")
])

In [10]:
strategy = fl.server.strategy.FedAdam(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=6,
    min_evaluate_clients=3,
    min_available_clients=6,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
    evaluate_metrics_aggregation_fn=weighted_average,
)

client_resources = None


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=6,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-15 17:27:28,349 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-15 17:27:33,061	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-15 17:27:35,182 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3896698060.0, 'memory': 7793396123.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'object_store_memory': 3896698060.0, 'memory': 7793396123.0, 'CPU': 2.0}
INFO flwr 2023-06-15 17:27:35,194 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-15 17:27:35,201 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-15 17:27:35,209 | server.py:88 | Evaluating initial para

[(5473, {'accuracy': 0.8770326972007751}), (5473, {'accuracy': 0.8825141787528992}), (5473, {'accuracy': 0.8825141787528992})]


DEBUG flwr 2023-06-15 17:33:57,623 | server.py:232 | fit_round 2 received 6 results and 0 failures
DEBUG:flwr:fit_round 2 received 6 results and 0 failures
DEBUG flwr 2023-06-15 17:33:57,666 | server.py:168 | evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 17:34:00,287 | server.py:182 | evaluate_round 2 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 3 results and 0 failures
DEBUG flwr 2023-06-15 17:34:00,294 | server.py:218 | fit_round 3: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 3: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.4425360858440399}), (5473, {'accuracy': 0.4525854289531708}), (5473, {'accuracy': 0.45276814699172974})]


DEBUG flwr 2023-06-15 17:36:58,663 | server.py:232 | fit_round 3 received 6 results and 0 failures
DEBUG:flwr:fit_round 3 received 6 results and 0 failures
DEBUG flwr 2023-06-15 17:36:58,704 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 17:37:01,302 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
DEBUG flwr 2023-06-15 17:37:01,306 | server.py:218 | fit_round 4: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 4: strategy sampled 6 clients (out of 6)


...

DEBUG flwr 2023-06-15 17:55:24,734 | server.py:232 | fit_round 9 received 6 results and 0 failures
DEBUG:flwr:fit_round 9 received 6 results and 0 failures
DEBUG flwr 2023-06-15 17:55:24,777 | server.py:168 | evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 17:55:28,771 | server.py:182 | evaluate_round 9 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 9 received 3 results and 0 failures
DEBUG flwr 2023-06-15 17:55:28,775 | server.py:218 | fit_round 10: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 10: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.974785327911377}), (5473, {'accuracy': 0.9725927114486694}), (5473, {'accuracy': 0.9736890196800232})]


DEBUG flwr 2023-06-15 17:58:27,091 | server.py:232 | fit_round 10 received 6 results and 0 failures
DEBUG:flwr:fit_round 10 received 6 results and 0 failures
DEBUG flwr 2023-06-15 17:58:27,134 | server.py:168 | evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 17:58:30,166 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-15 17:58:30,170 | server.py:147 | FL finished in 1854.946006155
INFO:flwr:FL finished in 1854.946006155
INFO flwr 2023-06-15 17:58:30,177 | app.py:218 | app_fit: losses_distributed [(1, 2.5589327812194824), (2, 13.335890134175619), (3, 1.1696898539861043), (4, 0.45720040798187256), (5, 0.36997583508491516), (6, 0.4300069312254588), (7, 0.2902422100305557), (8, 0.32072654366493225), (9, 0.33281315366427106), (10, 0.3950042128562927)]
INFO:flwr:app_fit: losses_distributed [(1

[(5473, {'accuracy': 0.9762470126152039}), (5473, {'accuracy': 0.9687557220458984}), (5473, {'accuracy': 0.9724100232124329})]


History (loss, distributed):
	round 1: 2.5589327812194824
	round 2: 13.335890134175619
	round 3: 1.1696898539861043
	round 4: 0.45720040798187256
	round 5: 0.36997583508491516
	round 6: 0.4300069312254588
	round 7: 0.2902422100305557
	round 8: 0.32072654366493225
	round 9: 0.33281315366427106
	round 10: 0.3950042128562927
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.8806870182355245), (2, 0.4492965539296468), (3, 0.9175954659779867), (4, 0.8785553177197775), (5, 0.9651623169581095), (6, 0.9635787804921468), (7, 0.9599244793256124), (8, 0.9456727107365926), (9, 0.9736890196800232), (10, 0.9724709192911783)]}